In [15]:
import torch
import time
import os
import pickle
from helper.dataloader import *
from helper.model import *
from helper.utils import *
from helper.losses import *
import torch.nn as nn
import torch.nn.functional as F
from helper.distance_map import get_dist_map
dtype = torch.float32
device = torch.device("cuda")

In [20]:
args_model_name = "uniref10_train_split_1_3200"
args_test_data = "uniref10_test_split_1_curate"
args_train_data = "uniref10_train_split_1"

args_hidden_dim = 512
args_out_dim = 128
id_ec_train, ec_id_dict_train = get_ec_id_dict(
    './data/' + args_train_data + '.csv')
id_ec_test, ec_id_dict_test = get_ec_id_dict(
    './data/' + args_test_data + '.csv')

model = LayerNormNet(args_hidden_dim, args_out_dim, device, dtype)
checkpoint = torch.load('./model/uniref10_split1_final.pth')


In [21]:
seed_everything()
model.zero_grad()
# the input to the model will be of size [bsz, (1+n_pos+n_neg), 1280]
n_pos = 1; n_neg = 50; bsz = 400; temp = 0.1
n_all = 1 + n_pos + n_neg
dummy_input = torch.randn(bsz, n_all, 1280).to(device, dtype)
model_emb = model(dummy_input)
loss = NPairLoss(model_emb, 0.1)
loss.backward()
print("loss is:", loss.item())
print("the grad of fc1 layer is:", model.fc1.weight.grad)

loss is: 5.79771089553833
the grad of fc1 layer is: tensor([[ 1.0353e-02,  4.3794e-03, -2.6401e-03,  ...,  5.6760e-03,
         -5.8931e-03, -1.0929e-03],
        [-2.1639e-03, -3.9893e-03,  6.3428e-03,  ..., -2.4620e-03,
         -3.2567e-03, -3.8886e-04],
        [ 2.2895e-05, -4.8278e-03,  5.4163e-04,  ..., -8.2139e-04,
          5.5884e-03,  6.6505e-03],
        ...,
        [ 3.5583e-03, -2.3453e-03,  1.7957e-03,  ...,  2.9575e-03,
         -3.4567e-03,  6.5465e-03],
        [-3.0378e-03,  7.3197e-04, -6.4718e-03,  ..., -7.4425e-04,
         -9.8990e-03,  7.8915e-03],
        [-2.3531e-03, -2.3591e-04, -8.8495e-04,  ..., -1.2460e-03,
         -3.3979e-03, -5.2351e-03]], device='cuda:0')
